In [1]:
#!pip install gspread oauth2client

In [76]:
#!jupyter labextension install jupyterlab-plotly

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
sns.set(rc={'figure.figsize':(15, 7)}, style='darkgrid')

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('my-day-analyze-f7d79d96a0ff.json', scope)
gc = gspread.authorize(credentials)

# открытие таблицы по ссылке или имени
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1HicRqzhoYOBCv0WRDP6UZdppTDSUFr4ifdWU_WzQq6c/edit?resourcekey#gid=898072817').sheet1


In [2]:
# получение всех значений в виде списка списков
data = worksheet.get_all_values()

# создание DataFrame из списка списков
df = pd.DataFrame(data[1:], columns=data[0])


In [3]:
from datetime import datetime
current_dateTime = datetime.now()

In [4]:
df.columns

Index(['Timestamp', 'Что делал?', 'Длительность (мин)',
       'Насколько ты удовлетворен тем, как сейчас проводишь время?', '',
       'Насколько ты устал?', 'Оцени свой уровень стресса ',
       'Насколько ты мотивирован?', 'Насколько ты сконцентрирован?',
       'Что можно сделать сейчас, чтобы почувствовать себя лучше? ',
       'Что тебе помогает сейчас чувствовать себя лучше?',
       'Что тебе сейчас нравится?', 'Что тебе сейчас не нравится?',
       'Что отвлекает?', 'Заметки: ',
       'Насколько ты хорошо себя чувствуешь физически'],
      dtype='object')

In [5]:
df['Timestamp'] = pd.to_datetime(df.Timestamp)

In [6]:
print(datetime.now().date())

2023-06-07


In [7]:
df_now = df[df.Timestamp.apply(lambda x: x.date()) == datetime.now().date()]

In [8]:
df_cur = df[df.Timestamp.apply(lambda x: x.date()) == datetime.strptime('2023-03-20', '%Y-%m-%d').date()]

In [9]:
numeric = ['Длительность (мин)',
       'Насколько ты удовлетворен тем, как сейчас проводишь время?',
       'Насколько ты устал?', 'Оцени свой уровень стресса ',
       'Насколько ты мотивирован?', 'Насколько ты сконцентрирован?',
       'Насколько ты хорошо себя чувствуешь физически']

In [10]:
df_cur.tail(4)

,Timestamp,Что делал?,Длительность (мин),"Насколько ты удовлетворен тем, как сейчас проводишь время?",,Насколько ты устал?,Оцени свой уровень стресса,Насколько ты мотивирован?,Насколько ты сконцентрирован?,"Что можно сделать сейчас, чтобы почувствовать себя лучше?",Что тебе помогает сейчас чувствовать себя лучше?,Что тебе сейчас нравится?,Что тебе сейчас не нравится?,Что отвлекает?,Заметки:,Насколько ты хорошо себя чувствуешь физически
210,2023-03-20 15:56:42,ML,45,6,,2,0,6,5,,,,,,,5
211,2023-03-20 20:48:21,Quantum mechanics,120,4,,1,1,5,6,,,,,,,5
212,2023-03-20 22:25:53,ML,20,7,,0,0,6,6,,,,,,,6
213,2023-03-20 23:24:21,Quantum mechanics,40,3,,0,2,3,5,,,,,,,5


In [11]:
cleaned_indexes = df_cur[numeric].apply(lambda x: np.vectorize(lambda x: x if x.isnumeric() else np.nan)(x)).dropna().index

In [12]:

clean_df = df.iloc[cleaned_indexes]
clean_df[numeric] = clean_df[numeric].astype(int)
clean_df['Длительность (мин)'] = clean_df['Длительность (мин)']/60

/home/tonyalpha/.local/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-12-cd7827f5501b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['Длительность (мин)'] = clean_df['Длительность (мин)']/60


In [13]:
data_for_pie = pd.DataFrame(clean_df.groupby(by=['Что делал?'])['Длительность (мин)'].sum()).reset_index()
#data_for_pie = data_for_pie.sort_values(by = 'Длительность (мин)')
total_time = clean_df['Длительность (мин)'].sum()
mean_concentration = clean_df['Насколько ты сконцентрирован?'].mean()
mean_motivation = clean_df['Насколько ты мотивирован?'].mean()

mean_exhaustion = clean_df['Насколько ты устал?'].mean()
mean_stress = clean_df['Оцени свой уровень стресса '].mean()
mean_motivation = clean_df['Насколько ты хорошо себя чувствуешь физически'].mean()

In [14]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

fig1 = px.pie(data_for_pie, values='Длительность (мин)', names='Что делал?')
fig1.update_traces(textinfo='label+percent+value', texttemplate='%{label} (%{percent})<br>%{value:.2f} h')
fig1.update_layout(
    title={
        'text': f'Total time: {total_time:.2f} hours Concentration = {mean_concentration:.2f}/10 Motivation = {mean_motivation:.2f}/10 Exhaustion = {mean_exhaustion:.2f}/10 Stress = {mean_stress:.2f}/10 ',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig1.show()

In [15]:
df.tail(10)

,Timestamp,Что делал?,Длительность (мин),"Насколько ты удовлетворен тем, как сейчас проводишь время?",,Насколько ты устал?,Оцени свой уровень стресса,Насколько ты мотивирован?,Насколько ты сконцентрирован?,"Что можно сделать сейчас, чтобы почувствовать себя лучше?",Что тебе помогает сейчас чувствовать себя лучше?,Что тебе сейчас нравится?,Что тебе сейчас не нравится?,Что отвлекает?,Заметки:,Насколько ты хорошо себя чувствуешь физически
238,2023-05-07 20:01:35,Work,30,6,,2,0,6,6,,,,,,,6
239,2023-05-07 23:39:35,NLP,30,7,,0,0,6,6,,,,,,,5
240,2023-05-08 09:44:01,NLP,30,6,,0,0,6,6,,,,,,,6
241,2023-05-08 09:44:33,NLP,45,6,,1,0,6,5,,,,,,,6
242,2023-05-08 17:28:29,Work,30,5,,0,0,5,5,,,,,,,6
243,2023-05-08 18:01:54,Work,30,6,,0,0,6,6,,,,,,,6
244,2023-05-21 17:00:31,NLP,30,6,,1,1,6,6,,,,,,,5
245,2023-05-21 17:10:02,Отдых,10,5,,0,0,6,6,,,,,,,5
246,2023-06-05 21:12:14,University Study things,30,4,,1,1,5,4,,,,,,,6
247,2023-06-06 13:12:55,Work,30,5,,0,1,6,5,,,,,,,6


In [21]:
#!pip3 install dash_bootstrap_components

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 2.5 MB/s eta 0:00:000:00:010:00:01
